In [2]:
path = "/home/jiahang/DiGress/chain_results/2024-04-13/11-28-34-protein-debug-2/protein-debug-2/{stage}"

In [3]:
import torch
import pandas as pd

In [4]:
import os
import pickle
stage = 'valid'

def read_metrics(stage, path):
    directory = path.format(stage=stage)
    metrics = []
    for filename in os.listdir(directory):
        if filename.endswith('.pkl'):
            file_path = os.path.join(directory, filename)
            
            # Read the .pkl file
            with open(file_path, 'rb') as file:
                metric = pickle.load(file)
            metrics.append(metric)
    return metrics

metrics = read_metrics('valid', path)

In [5]:
import numpy as np
import pandas as pd
def process_metrics(metrics):
    metric_list = ['acc', 'prec', 'rec', 'ce']
    metrics_Gt_1_Gt = {'acc': [], 'prec': [], 'rec': [], 'ce': []}
    metrics_X_Gt = {'acc': [], 'prec': [], 'rec': [], 'ce': []}

    for metric_epoch in metrics:
        for metric_name in metric_list:
            metrics_Gt_1_Gt[metric_name].append(metric_epoch['Gt_1_Gt'][metric_name])
            metrics_X_Gt[metric_name].append(metric_epoch['X_Gt'][metric_name])
    
    num_epochs, num_steps = len(metrics_Gt_1_Gt['acc']), len(metrics_Gt_1_Gt['acc'][0])
    for metric_name in metric_list:
        metrics_Gt_1_Gt[metric_name] = torch.concat(metrics_Gt_1_Gt[metric_name]).numpy()
        metrics_X_Gt[metric_name] = torch.concat(metrics_X_Gt[metric_name]).numpy()

    epoch_idx = [0]
    epoch_idx.extend(list(range(9, 100, 10)))
    epoch_idx = np.repeat(epoch_idx, num_steps)

    step_idx = list(range(0, 20)) * 11

    dict_Gt_1_Gt = {'epoch_idx': epoch_idx, 'step_idx': step_idx}
    dict_Gt_1_Gt.update(metrics_Gt_1_Gt)

    dict_X_Gt = {'epoch_idx': epoch_idx, 'step_idx': step_idx}
    dict_X_Gt.update(metrics_X_Gt)

    df_Gt_1_Gt = pd.DataFrame(dict_Gt_1_Gt).melt(id_vars=['epoch_idx', 'step_idx'], value_vars=['acc', 'prec', 'rec', 'ce'], var_name='metric')
    df_X_Gt = pd.DataFrame(dict_X_Gt).melt(id_vars=['epoch_idx', 'step_idx'], value_vars=['acc', 'prec', 'rec', 'ce'], var_name='metric')
    
    return df_Gt_1_Gt, df_X_Gt

df_Gt_1_Gt_a, df_X_Gt_a = process_metrics(metrics)

In [6]:
df_Gt_1_Gt

,epoch_idx,step_idx,acc,prec,rec,ce
0,0,0,0.485014,0.008275,0.484444,0.124103
1,0,1,0.486147,0.008667,0.506667,0.161404
2,0,2,0.489469,0.008949,0.520000,0.189110
3,0,3,0.492986,0.009313,0.537778,0.213402
4,0,4,0.499277,0.009277,0.528889,0.236708
...,...,...,...,...,...,...
215,99,15,0.636357,0.008444,0.346667,0.428584
216,99,16,0.654722,0.007999,0.311111,0.424512
217,99,17,0.677465,0.007484,0.271111,0.412154
218,99,18,0.705521,0.008339,0.275556,0.386056


# Gt_1_Gt

In [16]:
import plotly.express as px

fig = px.line(df_Gt_1_Gt_a, x="step_idx", y="value", color='epoch_idx', facet_col='metric')
fig.show()

# X_Gt

In [18]:
import plotly.express as px

fig = px.line(df_X_Gt_a, x="step_idx", y="value", color='epoch_idx', facet_col='metric')
fig.show()